### ปฏิบัติการครั้งที่ 9 กระบวนวิชา 229351 Statistical Learning for Data Science

คำชี้แจง

1. ให้เริ่มทำปฏิบัติการจาก colab notebook ที่กำหนดให้ จากนั้นตั้งค่า share ให้เหมาะสม
2. ส่ง link ใน Mango

ดาวน์โหลดข้อมูลการขายรถยนต์ใน link ข้างล่างนี้  
https://donlapark.pages.dev/229351/data/CarSales.csv  

In [4]:
!wget -O CarSales.csv https://donlapark.pages.dev/229351/data/CarSales.csv  

--2024-02-15 09:46:19--  https://donlapark.pages.dev/229351/data/CarSales.csv
Resolving donlapark.pages.dev (donlapark.pages.dev)... 172.66.47.56, 172.66.44.200, 2606:4700:310c::ac42:2cc8, ...
Connecting to donlapark.pages.dev (donlapark.pages.dev)|172.66.47.56|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1834 (1.8K) [text/csv]
Saving to: ‘CarSales.csv’

CarSales.csv        100%[===================>]   1.79K  --.-KB/s    in 0s      

2024-02-15 09:46:20 (20.3 MB/s) - ‘CarSales.csv’ saved [1834/1834]



In [5]:
# import module ที่ต้องใช้
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [6]:
# ฟังก์ชันที่เปลี่ยน string ให้เป็น datetime
def parser(dates):
  return pd.datetime.strptime(dates, '%Y-%m')

# parse_dates ชื่อของคอลัมน์ที่จะเปลี่ยนให้เป็น datetime
# index_col ชื่อของคอลัมน์ที่จะให้เป็น index
# date_parser ฟังก์ชันที่เปลี่ยน string ให้เป็น datetime
data = pd.read_csv('CarSales.csv', parse_dates=['Month'], 
                                        index_col='Month',
                                        date_parser=parser)

print (data.head())

/tmp/ipykernel_5782/3432910513.py:8: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  data = pd.read_csv('CarSales.csv', parse_dates=['Month'],


AttributeError: module 'pandas' has no attribute 'datetime'

In [ ]:
y = data['Sales'].astype('float32')

print(y)

In [ ]:
plt.figure(figsize=(12,6))
y.plot();

In [ ]:
def rmse(y_true,y_pred):
  # Inputs
  # y_true: actual value, y_pred: predicted values
  # Output: RMSE between y_true and y_pred
  return np.sqrt(np.mean(np.square(y_true-y_pred)))

# Train-Test split

In [ ]:
train = y[:-12]
test = y[-12:]

print(test)

## Test stationarity

### KPSS
- $H_0$: data is stationary
- $H_1$: data is **not** stationary


### ADF
- $H_0$: data is **not** stationary
- $H_1$: data is stationary


In [ ]:
from statsmodels.tsa.stattools import adfuller, kpss

def KPSStest(timeseries):
  kpsstest = kpss(timeseries)
  print('KPSS test')
  print('Test Statistic: ',kpsstest[0])
  print('p-value: ',kpsstest[1])
  print('Number of Observations Used: ',kpsstest[2])
  print('Critical Values: ',kpsstest[3])

def ADFtest(timeseries):
  adftest = adfuller(timeseries)
  print('ADF test')
  print('Test Statistic: ',adftest[0])
  print('p-value: ',adftest[1])
  print('#Lags Used: ',adftest[2])
  print('Number of Observations Used: ',adftest[3])
  print('Critical Values: ',adftest[4])

In [ ]:
KPSStest(y)
ADFtest(y)

In [ ]:
y_diff = y-y.shift()
y_diff.dropna(inplace=True)

KPSStest(y_diff)
ADFtest(y_diff)

# ARIMA($p,d,q$)

ลองพล็อต ACF และ PACF ของ time series **ที่ผ่าน difference แล้ว**

1. ถ้า ACF ลดลงไปเรื่อยๆ และ PACF หายไปหลังจาก lag ใด lag หนึ่ง แสดงว่าค่าของ lag นั้นเป็นค่าอันดับ AR (นั่นคือค่า $p$) ที่เหมาะสม
2. ถ้า PACF ลดลงไปเรื่อยๆ และ ACF หายไปหลังจาก lag ใด lag หนึ่ง แสดงว่าค่าของ lag นั้นเป็นค่าอันดับ MA (นั่นคือค่า $q$) ที่เหมาะสม

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

fig, ax = plt.subplots(2,1,figsize=(15, 10))

plot_acf(y_diff, lags=20, ax=ax[0]);
ax[0].axhline(y=-1.96/(len(y)-1)**.5, linestyle = '--')
ax[0].axhline(y=1.96/(len(y)-1)**.5, linestyle = '--')

plot_pacf(y_diff, lags=20, ax=ax[1]);
ax[1].axhline(y=-1.96/(len(y)-1)**.5, linestyle = '--')
ax[1].axhline(y=1.96/(len(y)-1)**.5, linestyle = '--')

In [ ]:
train

In [ ]:
from statsmodels.tsa.arima.model import ARIMA

arima = ARIMA(train, order=(0,1,0))


In [ ]:
arima_fit = arima.fit()
yhat = arima_fit.predict('1968-01-01', '1968-12-01')

In [ ]:
plt.figure(figsize=(12,6))
y.plot()
yhat.plot();

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
                                                       
sarima = SARIMAX(train, order=(0, 1, 0), seasonal_order=(0, 0, 1, 12)) 

sarima_fit = sarima.fit(optimized=True)
yhat = sarima_fit.predict('1968-01-01', '1968-12-01')

plt.figure(figsize=(12,6))
y.plot()
yhat.plot();

In [ ]:
print('RMSE:',rmse(test,yhat))

### ใช้ `auto_arima` จาก `pmdarima` เพื่อหา order ที่เหมาะสม

In [ ]:
!pip3 install pmdarima

In [ ]:
import pmdarima as pm

auto_fit = pm.auto_arima(train, start_p=0, start_q=0, max_p=3, max_q=3, d=0,
                             seasonal=True, m=12, start_P=0, start_Q=0, D=0,
                             information_criterion = 'aicc', # choose model with the lowest corrected AIC
                             trace = True,
                             error_action='ignore',  # don't want to know if an order does not work
                             suppress_warnings=True) # don't want convergence warnings

In [ ]:
auto_fit.summary()

In [ ]:
predictions = auto_fit.predict(12)
yhat = pd.Series(predictions,
                    index=test.index)

plt.figure(figsize=(12,6))
ax = y.plot()
yhat.plot(ax=ax);

In [ ]:
print('RMSE:',rmse(test,yhat))